# Hartree-Fock Self-Consistent Field Theory

In [1]:
"""Tutorial implementing a basic Hartree--Fock SCF program."""

__authors__ = "D. A. Sirianni"
__credits__ = "D. G. A. Smith"
__email__   = "sirianni.dom@gmail.com"

__copyright__ = "(c) 2014-2018, The Psi4NumPy Developers"
__license__   = "BSD-3-Clause"
__date__      = "05/22/2017"

## I. Theoretical Overview
In this tutorial, we will seek to introduce the theory and implementation of the quantum chemical method known as Hartree-Fock Self-Consistent Field Theory (HF-SCF) with restricted orbitals and closed-shell systems (RHF).  This theory seeks to solve the pseudo-eigenvalue matrix equation 

$$\sum_{\nu} F_{\mu\nu}C_{\nu i} = \epsilon_i\sum_{\nu}S_{\mu\nu}C_{\nu i}$$
$${\bf FC} = {\bf SC\epsilon},$$

called the Roothan equations, which can be solved self-consistently for the orbital coefficient matrix **C** to and the orbital energy eigenvalues $\epsilon_i$.  The Fock matrix, **F**, has elements $F_{\mu\nu}$ given (in the atomic orbital basis) as

$$F_{\mu\nu} = H_{\mu\nu} + 2(\mu\,\nu\left|\,\lambda\,\sigma)D_{\lambda\sigma} - (\mu\,\lambda\,\right|\nu\,\sigma)D_{\lambda\sigma},$$

where $D_{\lambda\sigma}$ is an element of the one-particle density matrix **D**, constructed from the orbital coefficient matrix **C**:

$$D_{\lambda\sigma} = C_{\sigma i}C_{\lambda i}$$

Formally, the orbital coefficient matrix **C** is a $N\times M$ matrix, where $N$ is the number of atomic basis functions, and $M$ is the total number of molecular orbitals.  Physically, this matrix describes the contribution of every atomic basis function (columns) to a particular molecular orbital (e.g., the $i^{\rm th}$ row).  The density matrix **D** is a square matrix describing the electron density contained in each orbital.  In the molecular orbital basis, the density matrix has elements

$$D_{pq} = \left\{
\begin{array}{ll}
2\delta_{pq} & p\; {\rm occupied} \\
0 & p\; {\rm virtual} \\
\end{array}\right .$$

The total RHF energy is given by

$$E^{\rm RHF}_{\rm total} = E^{\rm RHF}_{\rm elec} + E^{\rm BO}_{\rm nuc},$$

where $E^{\rm RHF}_{\rm elec}$ is the final electronic RHF energy, and $E^{\rm BO}_{\rm nuc}$ is the total nuclear repulsion energy within the Born-Oppenheimer approximation.  To compute the electronic energy, we may use the density matrix in the AO basis:

$$E^{\rm RHF}_{\rm elec} = (F_{\mu\nu} + H_{\mu\nu})D_{\mu\nu},$$

and the nuclear repulsion energy is simply

$$E^{\rm BO}_{\rm nuc} = \sum_{A>B}\frac{Z_AZ_B}{r_{AB}}$$

where $Z_A$ is the nuclear charge of atom $A$, and the sum runs over all unique nuclear pairs.

## II. Implementation

Using the above overview, let's write a RHF program using <span style="font-variant: small-caps"> Psi4 </span> and NumPy.  First, we need to import these Python modules: 

In [11]:
# ==> Import Psi4 & NumPy <==
import psi4
import numpy as np

Next, using what you learned in the previous tutorial module, set the following <span style="font-variant: small-caps"> Psi4 </span> and molecule options.

Memory & Output specifications:
- Give 500 Mb of memory to Psi4
- Set Psi4 output file to "output.dat"
- Set a variable `numpy_memory` to an acceptable amount of available memory for the working computer to use for storing tensors

Molecule definition:
- Define the "physicist's water molecule" (O-H bond length = 1.1 Angstroms, HOH bond angle = 104 degrees)
- Molecular symmetry C1

Computation options:
- basis set cc-pVDZ
- SCF type PK
- Energy convergence criterion to 0.00000001


In [12]:
# ==> Set Basic Psi4 Options <==
# Memory specification
psi4.set_memory(int(5e8))
numpy_memory = 2

# Set output file
psi4.core.set_output_file('output.dat', False)

# Define Physicist's water -- don't forget C1 symmetry!
#mol = psi4.geometry("""
#O
#H 1 1.0
#H 1 1.0 2 104.5
#symmetry c1
#""")
mol = psi4.geometry("""
N
N 1 1.1
symmetry c1
""")
# Set computation options
psi4.set_options({'basis': 'cc-pvdz',
                  'scf_type': 'pk',
                  'e_convergence': 1e-8})

Since we will be writing our own, iterative RHF procedure, we will need to define options that we can use to tweak our convergence behavior.  For example, if something goes wrong and our SCF doesn't converge, we don't want to spiral into an infinite loop.  Instead, we can specify the maximum number of iterations allowed, and store this value in a variable called `maxiter`.  Here are some good default options for our program:
~~~python
MAXITER = 40
E_conv = 1.0e-6
~~~
These are by no means the only possible values for these options, and it's encouraged to try different values and see for yourself how different choices affect the performance of our program.  For now, let's use the above as our default.

In [13]:
# ==> Set default program options <==
# Maximum SCF iterations
MAXITER = 40
# Energy convergence criterion
E_conv = 1.0e-6

Before we can build our Fock matrix, we'll need to compute the following static one- and two-electron quantities:

- Electron repulsion integrals (ERIs) **I** between our AOs
- Overlap matrix **S**
- Core Hamiltonian matrix **H**

Fortunately for us, we can do this using the machinery in <span style='font-variant: small-caps'> Psi4</span>.  In the first module, you learned about `psi4.core.Wavefunction` and `psi4.core.MintsHelper` classes.  In the cell below, use these classes to perform the following:

1. Create Class Instances

    a. Build a wavefunction for our molecule and basis set
    
    b. Create an instance of the `MintsHelper` class with the basis set for the wavefunction

2. Build overlap matrix, **S**

    a. Get the AO overlap matrix from `MintsHelper`, and cast it into a NumPy array
    
    b. Get the number of AO basis functions and number of doubly occupied orbitals from S and the wavefunciton

3. Compute ERI Tensor, **I**

    a. Get ERI tensor from `MintsHelper`, and cast it into a NumPy array

4. Build core Hamiltonian, **H**

    a. Get AO kinetic energy matrix from `MintsHelper`, and cast it into a NumPy array

    b. Get AO potential energy matrix from `MintsHelper`, and cast it into a NumPy array

    c. Build core Hamiltonian from kinetic & potential energy matrices

In [14]:
# ==> Compute static 1e- and 2e- quantities with Psi4 <==
# Class instantiation
wfn = psi4.core.Wavefunction.build(mol, psi4.core.get_global_option('basis'))
mints = psi4.core.MintsHelper(wfn.basisset())

# Overlap matrix
S = np.asarray(mints.ao_overlap())

# Number of basis Functions & doubly occupied orbitals
nbf = S.shape[0]
ndocc = wfn.nalpha()

print('Number of occupied orbitals: %3d' % (ndocc))
print('Number of basis functions: %3d' % (nbf))

# Memory check for ERI tensor
I_size = (nbf**4) * 8.e-9
print('\nSize of the ERI tensor will be {:4.2f} GB.'.format(I_size))
memory_footprint = I_size * 1.5
if I_size > numpy_memory:
    psi4.core.clean()
    raise Exception("Estimated memory utilization (%4.2f GB) exceeds allotted memory \
                     limit of %4.2f GB." % (memory_footprint, numpy_memory))

# Build ERI Tensor
I = np.asarray(mints.ao_eri())

# Build core Hamiltonian
T = np.asarray(mints.ao_kinetic())
V = np.asarray(mints.ao_potential())
H = T + V

print(H)

Number of occupied orbitals:   7
Number of basis functions:  28

Size of the ERI tensor will be 0.00 GB.
[[-2.77943536e+01  6.66403536e+00 -4.94760972e+00 -1.50796599e-01
   0.00000000e+00  0.00000000e+00 -3.02044502e-02  0.00000000e+00
   0.00000000e+00 -1.78199460e-02  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00 -2.01897296e-02 -6.98977569e-01
  -1.96331488e+00  1.35994905e+00  0.00000000e+00  0.00000000e+00
   3.66219623e+00  0.00000000e+00  0.00000000e+00 -3.20045003e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 6.66403536e+00 -1.03826457e+01 -5.61984398e+00 -8.59176412e-01
   0.00000000e+00  0.00000000e+00 -6.14060079e-01  0.00000000e+00
   0.00000000e+00 -4.74210425e-01  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00 -6.98977569e-01 -2.04612135e+00
  -3.17922145e+00  2.79385014e+00  0.00000000e+00  0.00000000e+00
   4.06598239e+00  0.00000000e+00  0.00000000e+00 -2.46311654e+00
   0.00000000e+00  0.00000000e+00  0

The Roothan equations

$${\bf FC} = {\bf SC\epsilon}$$

are only *pseudo*-eigenvalue equations due to the presence of the overlap matrix **S** on the right hand side of the equation.  Normally, the AO basis set will not be orthonormal, so the overlap matrix **S** will not be unity and therefore cannot be ignored.  Let's check to see whether our AO basis is orthonormal:

In [15]:
# ==> Inspecting S for AO orthonormality <==
hope = np.allclose(S, np.eye(S.shape[0]))
print('\nDo we have any hope that our AO basis is orthonormal? %s!' % (hope))


Do we have any hope that our AO basis is orthonormal? False!


Just as we'd expected -- looks like we can't ignore the AO overlap matrix.  Therefore, the Fock matrix **F** cannot simply be diagonalized to solve for the orbital coefficient matrix **C**.  There is still hope, however!  We can overcome this issue by transforming the AO basis so that all of our basis functions are orthonormal.  In other words, we seek a matrix **A** such that the transformation 

$${\bf A}^{\dagger}{\bf SA} = {\bf 1}$$

One method of doing this is called *symmetric orthogonalization*, which lets ${\bf A} = {\bf S}^{-1/2}$.  Then, 

$${\bf A}^{\dagger}{\bf SA} = {\bf S}^{-1/2}{\bf SS}^{-1/2} = {\bf S}^{-1/2}{\bf S}^{1/2} = {\bf S}^0 = {\bf 1},$$

and we see that this choice for **A** does in fact yield an orthonormal AO basis.  In the cell below, construct this transformation matrix using <span style='font-variant: small-caps'> Psi4</span>'s built-in `Matrix` class member function `power()` just like the following:
~~~python
A = mints.ao_overlap()
A.power(-0.5, 1.e-16)
A = np.asarray(A)
~~~

In [16]:
# ==> Construct AO orthogonalization matrix A <==
A = mints.ao_overlap()
A.power(-0.5, 1.e-16)
A = np.asarray(A)

print(A)
# Check orthonormality
S_p = A.dot(S).dot(A)
new_hope = np.allclose(S_p, np.eye(S.shape[0]))

if new_hope:
    print('There is a new hope for diagonalization!')
else:
    print("Whoops...something went wrong. Check that you've correctly built the transformation matrix.")

[[ 1.17017156e+00  4.39173671e-01 -4.42532230e-01  8.91940299e-02
   7.98844772e-17  5.42767004e-17 -1.25680292e-01  3.80226662e-16
  -1.12566193e-17  5.90048297e-03 -2.28578020e-16 -5.43972934e-17
   2.18837423e-16  0.00000000e+00  3.89871748e-03 -1.15783021e-02
   1.28954255e-01  4.66800183e-02  1.56434413e-16  1.54678211e-16
   3.23424743e-02 -7.73320252e-17 -1.18746783e-16 -1.67158471e-01
   2.84487388e-16 -3.49528756e-18 -7.23726875e-17  0.00000000e+00]
 [ 4.39173671e-01  1.73679899e+00 -9.02666302e-01  2.01939476e-01
   2.50668850e-17  8.09606517e-17 -1.94898920e-01  2.45027605e-16
   6.56783566e-17  4.70030939e-02  3.54313136e-17 -1.14935374e-16
   6.77215848e-17  0.00000000e+00 -1.15783021e-02 -7.70429240e-02
   1.82548673e-01  1.58404375e-01  2.58924611e-16  2.71179658e-16
   1.07604281e-01 -1.36654419e-17 -2.00246410e-16 -3.40806911e-01
   2.87807948e-16 -4.50722577e-18 -2.08084528e-16  0.00000000e+00]
 [-4.42532230e-01 -9.02666302e-01  5.68193307e+00  2.85833185e-01
  -7.495

The drawback of this scheme is that we would now have to either re-compute the ERI and core Hamiltonian tensors in the newly orthogonal AO basis, or transform them using our **A** matrix (both would be overly costly, especially transforming **I**).  On the other hand, substitute ${\bf C} = {\bf AC}'$ into the Roothan equations:

\begin{align}
{\bf FAC'} &= {\bf SAC}'{\bf \epsilon}\\
{\bf A}^{\dagger}({\bf FAC}')&= {\bf A}^{\dagger}({\bf SAC}'){\bf \epsilon}\\
({\bf A}^{\dagger}{\bf FA}){\bf C}'&= ({\bf A}^{\dagger}{\bf SA}){\bf C}'{\bf \epsilon}\\
{\bf F}'{\bf C}' &= {\bf 1C}'{\bf \epsilon}\\
{\bf F}'{\bf C}' &= {\bf C}'{\bf \epsilon}\\
\end{align}

Clearly, we have arrived at a canonical eigenvalue equation.  This equation can be solved directly for the transformed orbital coefficient matrix ${\bf C}'$ by diagonalizing the transformed Fock matrix, ${\bf F}'$, before transforming ${\bf C}'$ back into the original AO basis with ${\bf C} = {\bf AC}'$.  

Before we can get down to the business of using the Fock matrix **F** to compute the RHF energy, we first need to compute the orbital coefficient **C** matrix.  But, before we compute the **C** matrix, we first need to build **F**.  Wait...hold on a second.  Which comes first, **C** or **F**?  Looking at the Roothan equations more closely, we see that that both sides depend on the **C** matrix, since **F** is a function of the orbitals:


$${\bf F}({\bf C}){\bf C} = {\bf SC\epsilon}\,;\;\;F_{\mu\nu} = H_{\mu\nu} + 2(\mu\,\nu\mid\lambda\,\sigma)C_{\sigma i}C_{\lambda i} - (\mu\,\lambda\,\mid\nu\,\sigma)C_{\sigma i}C_{\lambda i}.$$

Therefore technically, *neither* **F** nor **C** can come first!  In order to proceed, we instead begin with a *guess* for the Fock matrix, from which we obtain a guess at the **C** matrix.  Without orbital coefficients (and therefore without electron densities), the most logical starting point for obtaining a guess at the Fock matrix is to begin with the only component of **F** that does *not* involve densities: the core Hamiltonian, **H**.  Below, using the NumPy `np.linalg.eigh()` function, obtain coefficient and density matrices using the core guess:

1. Obtain ${\bf F}'$ by transforming the core Hamiltonian with the ${\bf A}$ matrix
2. Diagonalize the transformed Fock matrix for $\epsilon$ and ${\bf C}'$
3. Use doubly-occupied slice of coefficient matrix to build density matrix

In [17]:
# ==> Compute C & D matrices with CORE guess <==
# Transformed Fock matrix
F_p = A.dot(H).dot(A)

# Diagonalize F_p for eigenvalues & eigenvectors with NumPy
e, C_p = np.linalg.eigh(F_p)


# Transform C_p back into AO basis
C = A.dot(C_p)
#print(C)
# Grab occupied orbitals
C_occ = C[:, :ndocc]

# Build density matrix from occupied orbitals
D = np.einsum('pi,qi->pq', C_occ, C_occ, optimize=True)
print(D)

[[ 1.05306058e+00  2.10654840e-01  1.25960158e-01  3.21791495e-01
  -1.57919806e-15  1.26000822e-15  4.04466068e-02  1.61917655e-15
   1.07881246e-16  8.39702183e-02 -7.76548781e-16  1.10686016e-16
   1.85859733e-16  6.41941020e-31 -6.95691526e-02 -1.98726789e-01
  -1.17278527e-01  2.51418009e-01  2.97161441e-15  6.99634519e-16
   5.18779060e-02 -4.97252413e-16 -5.87638424e-16 -6.91319923e-02
  -9.57563042e-17  2.85364845e-16  2.88816722e-16  2.90004989e-30]
 [ 2.10654840e-01  8.48547331e-01  2.75204161e-01  1.01152259e+00
  -5.87665524e-15  3.02399580e-15  9.05857990e-02  1.21774440e-15
   5.21519110e-16  2.65410813e-01 -1.22731674e-15  2.71117645e-16
  -6.70082616e-16  1.65060536e-30 -1.98726789e-01 -5.25621748e-01
  -2.98436405e-01  6.70840671e-01  6.33422357e-15  1.44670069e-15
   1.39387218e-01  7.24650569e-16 -1.43825503e-15 -1.87958525e-01
  -1.52360608e-15  8.94682451e-16  8.53804249e-16  8.58708308e-30]
 [ 1.25960158e-01  2.75204161e-01  1.23228886e-01  3.41776084e-01
  -9.525

The final quantity we need to compute before we can proceed with our implementation of the SCF procedure is the Born-Oppenheimer nuclear repulsion energy, $E^{\rm BO}_{\rm nuc}$.  We could use the expression given above in $\S$1, however we can also obtain this value directly from <span style='font-variant: small-caps'> Psi4</span>'s `Molecule` class.  In the cell below, compute the nuclear repulsion energy using either method. 

In [18]:
# ==> Nuclear Repulsion Energy <==
E_nuc = mol.nuclear_repulsion_energy()

Within each SCF iteration, we'll have to perform a number of tensor contractions when building the Fock matrix, computing the total RHF energy, and performing several transformations.  Since the computationl expense of this process is related to the number of unique indices, the most intensive step of computing the total electronic energy will be performing the four-index contractions corresponding to building Coulomb and Exchange matrices **J** and **K**, with elements

\begin{align}
J[D_{\lambda\sigma}]_{\mu\nu} &= (\mu\,\nu\mid\lambda\,\sigma)D_{\lambda\sigma}\\
K[D_{\lambda\sigma}]_{\mu\nu} &= (\mu\,\lambda\mid\nu\,\sigma)D_{\lambda\sigma},
\end{align}

when building the Fock matrix.  Fortunately, once **J** and **K** have been built, the Fock matrix may be computed as a simple matrix addition, instead of element-wise:

$$ {\bf F} = {\bf H} + 2{\bf J} - {\bf K}.$$

Formation of the **J** and **K** matrices will be the most expensive step of the RHF procedure, scaling with respect to the number of AOs as ${\cal O}(N^4)$.  Strategies for building these marices efficiently, as well as different methods for handling these tensor contractions, will be discussed in greater detail in tutorials 2c and 2d in this module, respectively. 

Let's now write our SCF iterations according to the following algorithm:

#### Algorithm 1: SCF Iteration
for scf_iter less than MAXITER, do:
1. Build Fock matrix
    - Build the Coulomb matrix **J** 
    - Build the Exchange matrix **K** 
    - Form the Fock matrix
2. RHF Energy
    - Compute total RHF energy   
    - If change in RHF energy less than E_conv, break    
    - Save latest RHF energy as E_old
3. Compute new orbital guess
    - Transform Fock matrix to orthonormal AO basis    
    - Diagonalize ${\bf F}'$ for $\epsilon$ and ${\bf C}'$    
    - Back transform ${\bf C}'$ to AO basis    
    - Form **D** from occupied orbital slice of **C**


In [19]:
# ==> SCF Iterations <==
# Pre-iteration energy declarations
SCF_E = 0.0
E_old = 0.0

print('==> Starting SCF Iterations <==\n')

# Begin Iterations
for scf_iter in range(1, MAXITER + 1):
    # Build Fock matrix
    J = np.einsum('pqrs,rs->pq', I, D, optimize=True)
    K = np.einsum('prqs,rs->pq', I, D, optimize=True)
    F = H + 2*J - K
    
    # Compute RHF energy
    SCF_E = np.einsum('pq,pq->', (H + F), D, optimize=True) + E_nuc
    print('SCF Iteration %3d: Energy = %4.16f dE = % 1.5E' % (scf_iter, SCF_E, SCF_E - E_old))
    
    # SCF Converged?
    if (abs(SCF_E - E_old) < E_conv):
        break
    E_old = SCF_E
    
    # Compute new orbital guess
    F_p =  A.dot(F).dot(A)
    e, C_p = np.linalg.eigh(F_p)
    C = A.dot(C_p)
    C_occ = C[:, :ndocc]
    D = np.einsum('pi,qi->pq', C_occ, C_occ, optimize=True)
    
    # MAXITER exceeded?
    if (scf_iter == MAXITER):
        psi4.core.clean()
        raise Exception("Maximum number of SCF iterations exceeded.")

# Post iterations
print('\nSCF converged.')
print('Final RHF Energy: %.8f [Eh]' % (SCF_E))

==> Starting SCF Iterations <==

SCF Iteration   1: Energy = -99.9245105453145754 dE = -9.99245E+01
SCF Iteration   2: Energy = -106.5782011546040309 dE = -6.65369E+00
SCF Iteration   3: Energy = -108.7929789316870739 dE = -2.21478E+00
SCF Iteration   4: Energy = -108.9449851100444704 dE = -1.52006E-01
SCF Iteration   5: Energy = -108.9532753806098100 dE = -8.29027E-03
SCF Iteration   6: Energy = -108.9537639570678920 dE = -4.88576E-04
SCF Iteration   7: Energy = -108.9537940394515232 dE = -3.00824E-05
SCF Iteration   8: Energy = -108.9537960746021241 dE = -2.03515E-06
SCF Iteration   9: Energy = -108.9537962270221243 dE = -1.52420E-07

SCF converged.
Final RHF Energy: -108.95379623 [Eh]


Congratulations! You've written your very own Restricted Hartree-Fock program!  Finally, let's check your final RHF energy against <span style='font-variant: small-caps'> Psi4</span>:

In [20]:
# Compare to Psi4
SCF_E_psi = psi4.energy('SCF')
psi4.compare_values(SCF_E_psi, SCF_E, 6, 'SCF Energy')

    SCF Energy............................................................................PASSED


True

## References
1. [[Szabo:1996](http://store.doverpublications.com/0486691861.html)] A. Szabo and N. S. Ostlund, *Modern Quantum Chemistry*, Introduction to Advanced Electronic Structure Theory. Courier Corporation, 1996.
2. [[Levine:2000](https://books.google.com/books?id=80RpQgAACAAJ&dq=levine%20quantum%20chemistry%205th%20edition&source=gbs_book_other_versions)] I. N. Levine, *Quantum Chemistry*. Prentice-Hall, New Jersey, 5th edition, 2000.
3. [[Helgaker:2000](https://books.google.com/books?id=lNVLBAAAQBAJ&pg=PT1067&dq=helgaker+molecular+electronic+structure+theory&hl=en&sa=X&ved=0ahUKEwj37I7MkofUAhWG5SYKHaoPAAkQ6AEIKDAA#v=onepage&q=helgaker%20molecular%20electronic%20structure%20theory&f=false)] T. Helgaker, P. Jorgensen, and J. Olsen, *Molecular Electronic Structure Theory*, John Wiley & Sons Inc, 2000.